In [1]:
#이상치제거6파일에서 밑에서부터 620개를 선택
from colormath.color_objects import LabColor, sRGBColor 
from colormath.color_conversions import convert_color
from colormath.color_diff import delta_e_cmc
import numpy as np
import pandas as pd 
from colormath.color_objects import sRGBColor, XYZColor, LabColor
from colormath.color_conversions import convert_color
import pandas as pd
import math
import numpy as np
import numpy as np
import pandas as pd 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from PIL import Image
from matplotlib import pyplot as plt
import copy
from sklearn.preprocessing import LabelEncoder
import pandas.core.algorithms as algos
from pandas import Series
#from pandas import tools
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
from sklearn.preprocessing import LabelEncoder,StandardScaler,OneHotEncoder,MinMaxScaler
from sklearn.svm import SVR
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from sklearn.linear_model import LinearRegression
from tensorflow import keras
import tensorflow_addons as tfa
import pandas as pd
import numpy as np
from tensorflow.keras import backend as K
# gpu 상태 확인 및 메모리 사용만큼 할당
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
from colormath.color_objects import LabColor, sRGBColor 
from colormath.color_conversions import convert_color
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib._color_data as mcd
import matplotlib.patches as mpatch
import webcolors
from colormath.color_objects import LabColor, sRGBColor 
from colormath.color_conversions import convert_color 
from colormath.color_diff import delta_e_cie2000
from colormath import color_diff_matrix
import matplotlib.colors as mcolors

from math import sqrt
from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xlrd
import openpyxl
from pandasql import sqldf

In [2]:
def paste_python(aa,a):
    for i in range(len(aa)):
        aa[i]=a+aa[i]
    return(aa)
def paste_python2(a,aa):
    for i in range(len(aa)):
        aa[i]=aa[i]+a
    return(aa)
# lab를 rgb로 변환하는 함수
def lab_to_rgb(data):
    r_list=[]
    g_list=[]
    b_list=[]
    for i in range(len(data)):
        lab=LabColor(data[data.columns[0]][data.index[i]],data[data.columns[1]][data.index[i]],data[data.columns[2]][data.index[i]])#lab좌표
        rgb=convert_color(lab,sRGBColor)#rgb좌표
        rgb_list=[color for color in rgb.get_value_tuple()]
        r_list.append(rgb_list[0])
        g_list.append(rgb_list[1])
        b_list.append(rgb_list[2])
    df = pd.DataFrame(
                {'r' : r_list,
                 'g' : g_list,
                 'b' : b_list}
                )
    df.index=data.index
    return(df)
def color_diff(pred_TRANS,Y_TRANS):
    diff = []
    for i in Y_TRANS.index:
        pred = pred_TRANS.loc[i]
        true = Y_TRANS.loc[i]
        p_lab = LabColor(pred[0],pred[1],pred[2])
        t_lab = LabColor(true[0],true[1],true[2])
        delta = delta_e_cmc(p_lab , t_lab)
        diff.append(delta)
    diff=pd.DataFrame(diff)
    diff.index=Y_TRANS.index
    return(diff)
class AttentionLayer(tf.keras.layers.Layer):
    # 첨 class 선언신 변수생성
    def __init__(self,input_units, num_unit,output_unit,lams,**kwargs,
                ):
        super(AttentionLayer, self).__init__(**kwargs)
        self.input_units = input_units
        self.num_unit = num_unit
        self.output_unit = output_unit
        self.lambdas = lams
        
        
        
        
    def get_config(self):
        
        config = super().get_config().copy()
        config.update({
            'input_units': self.input_units,
            'num_unit': self.num_unit,
            'output_unit': self.output_unit,
            'lambdas': self.lambdas,
        })
        return config 
        
    def build(self, input_shape):  
        self.kernel1 = self.add_weight("kernel1",
                    initializer = tf.keras.initializers.glorot_normal(),
                    regularizer = tf.keras.regularizers.l2(self.lambdas),
                    shape =[self.input_units,self.num_unit])
        self.bias1 = self.add_weight("bias1",initializer = tf.zeros_initializer(),shape=[1,self.num_unit])
        
        
        
        self.kernel2 = self.add_weight("kernel2",
                    initializer = tf.keras.initializers.glorot_normal(),
                    regularizer = tf.keras.regularizers.l2(self.lambdas),
                    shape = [self.num_unit,self.output_unit])
        self.bias2 = self.add_weight("bias2",initializer = tf.zeros_initializer(),shape=[1,self.output_unit])
        
                    
        
    def call(self, inputs):
        shape = [ tf.shape(inputs)[k] for k in range(len(inputs.shape))]
        
        shape_list = [shape[0]] + [1 for i in range(len(inputs.shape)-1)]
        kernel1_ = tf.tile(tf.expand_dims(self.kernel1,0),shape_list)
        kernel2_ = tf.tile(tf.expand_dims(self.kernel2,0),shape_list)
        
        h1 = tf.matmul(inputs,kernel1_) + self.bias1
        h1 = tf.nn.tanh(h1)
        out = tf.matmul(h1,kernel2_) + self.bias2
        out = tf.nn.softmax(out,axis=1)
        flat_output_total = tf.multiply(inputs,tf.expand_dims(out[:,:,0],axis=2))
        flat_output_total = tf.reduce_sum(flat_output_total,axis=1)
        for i in range(1,self.output_unit):
            flat_output= tf.multiply(inputs,tf.expand_dims(out[:,:,i],axis=2))
            flat_output = tf.reduce_sum(flat_output,axis=1)
            flat_output_total = tf.concat([flat_output_total,flat_output],axis=1)
        
        return flat_output_total
    # CIE94 Keras Custom Loss
    def cie94(y_true, y_pred):
        """Calculate color difference by using CIE94 formulae

        See http://en.wikipedia.org/wiki/Color_difference or
        http://www.brucelindbloom.com/index.html?Eqn_DeltaE_CIE94.html.

        cie94(rgb2lab((255, 255, 255)), rgb2lab((0, 0, 0)))
        >>> 58.0
        cie94(rgb2lab(rgb(0xff0000)), rgb2lab(rgb('#ff0000')))
        >>> 0.0
        """

        L1, a1, b1 = y_true[:,0],y_true[:,1],y_true[:,2]
        L2, a2, b2 = y_pred[:,0],y_pred[:,1],y_pred[:,2]


        C1 = K.sqrt(K.pow(a1,2) + K.pow(b1,2))
        C2 = K.sqrt(K.pow(a2,2) + K.pow(b2,2))
        delta_L = L1 - L2
        delta_C = C1 - C2
        delta_a = a1 - a2
        delta_b = b1 - b2
        delta_H_square = K.pow(delta_a,2) + K.pow(delta_b,2) - K.pow(delta_C,2)
        loss_a = (K.sqrt(K.pow(delta_L,2)
                + K.pow(delta_C,2) / K.pow(1.0 + 0.045 * C1,2)
                + delta_H_square / K.pow(1.0 + 0.015 * C1,2)))
        return loss_a
#전처리_조제=dataframe
#전처리_조제명=dataframe_names
#전처리_조제량=dataframe_amount
def pivot_PET(dataframe):
    whe=[]
    for i in dataframe.columns:
        if '명' in i:
            whe.append(1)
        else:
            whe.append(0)
    whe=np.array(whe)
    
    dataframe_names=dataframe.columns[np.where(whe==1)]
    dataframe_amount=dataframe.columns[np.where(whe==0)]
    
    dataframe_names_df=dataframe[dataframe_names]
    dataframe_amount_df=dataframe[dataframe_amount]

    
    dataframe_names_unique=[]
    for i in dataframe_names:
        dataframe_names_unique.extend(dataframe[i].unique())

    col_names=pd.DataFrame(dataframe_names_unique)[0].unique()
    col_names=col_names[~pd.isnull(col_names)]
    dataframe_reshape=pd.DataFrame(index=dataframe.index,columns=col_names)
    
    for i in range(len(dataframe_reshape)):
        for j in range(len(dataframe_names_df.columns)):
            if pd.isna(dataframe_names_df.loc[i][j])==False:
                dataframe_reshape.loc[i][np.where(dataframe_reshape.columns==dataframe_names_df.loc[i][j])[0][0]]=dataframe_amount_df.loc[i][j]#[0][0]는고정
            else:
                continue
    return(dataframe_reshape)

In [3]:
# 데이터 셋 불러오기 (KS값)
data = pd.read_excel('C:/Users/leewa/Desktop/e-MFG/다이텍/데이터/원본데이터/1월/기초DB_통합자료(다이텍,부산대)_공정분야v230101_성균관대발송분 (3)')
data

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 432,내광성 시험 전 검사,Unnamed: 434,Unnamed: 435,내광성 시험 후 검사,Unnamed: 437,Unnamed: 438,Unnamed: 439,Unnamed: 440,Unnamed: 441
0,NaN,번호,시료번호,내광성시험분석,접수번호No.,업체명,염색법,기계분류ID,기계분류,원료ID,...,건식마찰착색성,L*,a*,b*,L*,a*,b*,내광성등급,색차,비고
1,구분,-,-,-,-,-,-,-,-,-,...,물성,물성,물성,물성,물성,물성,물성,물성,물성,NaN
2,단위,-,-,NaN,-,-,1욕 or 2욕,-,-,-,...,급,-,-,-,-,-,-,급,△E,NaN
3,시간,1273,209,카트리(2차),2021-10-18 00:00:00,다이텍연구원,1,P-DY09,고압 래피드 염색기,RAW-029,...,4,47.99,60.25,20.79,52.926667,49.953333,16.43,1,12.22,NaN
4,시간,1274,209,카트리(2차),2021-10-18 00:00:00,다이텍연구원,1,P-DY09,고압 래피드 염색기,RAW-029,...,4,47.99,60.25,20.79,52.926667,49.953333,16.43,1,12.22,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3168,시간_검증1차,5602,798,1차년도_다이텍,2022-11-15 00:00:00,다이텍연구원,1,P-DY09,고압 래피드 염색기,RAW-029,...,4,25.73,9.35,-23.05,31.196667,7.793333,-15.956667,1.5,9.09,NaN
3169,시간_검증1차,5603,799,1차년도_다이텍,2022-11-15 00:00:00,다이텍연구원,1,P-DY09,고압 래피드 염색기,RAW-029,...,4,25.72,9.16,-22.72,30.626667,7.673333,-16.13,1.5,8.36,NaN
3170,시간_검증1차,5604,799,1차년도_다이텍,2022-11-15 00:00:00,다이텍연구원,1,P-DY09,고압 래피드 염색기,RAW-029,...,4,25.72,9.16,-22.72,30.626667,7.673333,-16.13,1.5,8.36,NaN
3171,시간_검증1차,5605,800,1차년도_다이텍,2022-11-15 00:00:00,다이텍연구원,1,P-DY09,고압 래피드 염색기,RAW-029,...,4,25.73,9.22,-22.65,30.413333,7.65,-16.283333,1.5,8.06,NaN


In [4]:
data=pd.read_excel('C:/Users/leewa/Desktop/1월 원본데이터.xlsx')#,encoding='euc-kr')
data=data[data.columns[1:]]
data=data.drop([1,2,],axis=0)
data=data.reset_index()
data=data[data.columns[1:]]
data

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,경사,...,Unnamed: 432,내광성 시험 전 검사,Unnamed: 434,Unnamed: 435,내광성 시험 후 검사,Unnamed: 437,Unnamed: 438,Unnamed: 439,Unnamed: 440,Unnamed: 441
0,번호,시료번호,내광성시험분석,접수번호No.,업체명,염색법,기계분류ID,기계분류,원료ID,소재명,...,건식마찰착색성,L*,a*,b*,L*,a*,b*,내광성등급,색차,비고
1,1273,209,카트리(2차),2021-10-18 00:00:00,다이텍연구원,1,P-DY09,고압 래피드 염색기,RAW-029,PET 재생 섬유,...,4,47.99,60.25,20.79,52.926667,49.953333,16.43,1,12.22,NaN
2,1274,209,카트리(2차),2021-10-18 00:00:00,다이텍연구원,1,P-DY09,고압 래피드 염색기,RAW-029,PET 재생 섬유,...,4,47.99,60.25,20.79,52.926667,49.953333,16.43,1,12.22,NaN
3,1275,210,카트리(2차),2021-10-18 00:00:00,다이텍연구원,1,P-DY09,고압 래피드 염색기,RAW-029,PET 재생 섬유,...,4,46.64,61.99,23.04,49.993333,52.37,17.73,1,11.49,NaN
4,1276,210,카트리(2차),2021-10-18 00:00:00,다이텍연구원,1,P-DY09,고압 래피드 염색기,RAW-029,PET 재생 섬유,...,4,46.64,61.99,23.04,49.993333,52.37,17.73,1,11.49,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3166,5602,798,1차년도_다이텍,2022-11-15 00:00:00,다이텍연구원,1,P-DY09,고압 래피드 염색기,RAW-029,PET 재생 섬유,...,4,25.73,9.35,-23.05,31.196667,7.793333,-15.956667,1.5,9.09,NaN
3167,5603,799,1차년도_다이텍,2022-11-15 00:00:00,다이텍연구원,1,P-DY09,고압 래피드 염색기,RAW-029,PET 재생 섬유,...,4,25.72,9.16,-22.72,30.626667,7.673333,-16.13,1.5,8.36,NaN
3168,5604,799,1차년도_다이텍,2022-11-15 00:00:00,다이텍연구원,1,P-DY09,고압 래피드 염색기,RAW-029,PET 재생 섬유,...,4,25.72,9.16,-22.72,30.626667,7.673333,-16.13,1.5,8.36,NaN
3169,5605,800,1차년도_다이텍,2022-11-15 00:00:00,다이텍연구원,1,P-DY09,고압 래피드 염색기,RAW-029,PET 재생 섬유,...,4,25.73,9.22,-22.65,30.413333,7.65,-16.283333,1.5,8.06,NaN


In [5]:
#경사 전까지 컬렴명변경
list_col1=list(data.loc[0][np.where(data.columns=='Unnamed: 1')[0][0]:np.where(data.columns=='경사')[0][0]])
list_col1.extend(data.columns[np.where(data.columns=='경사')[0][0]:])
data.columns=list_col1
np.array(data.columns)
#경사이후, 위사이전까지 컬럼명 변경
list_col1=list(data.loc[0][np.where(data.columns=='번호')[0][0]:np.where(data.columns=='경사')[0][0]])
list_col1.extend(paste_python(list(data.loc[0][np.where(data.columns=='경사')[0][0]:np.where(data.columns=='위사')[0][0]]),'경사_'))
list_col1.extend(data.columns[np.where(data.columns=='위사')[0][0]:])
data.columns=list_col1
np.array(data.columns)
#위사이후, 제직이전까지 컬럼명 변경
list_col1=list(data.columns[np.where(data.columns=='번호')[0][0]:np.where(data.columns=='위사')[0][0]])
list_col1.extend(paste_python(list(data.loc[0][np.where(data.columns=='위사')[0][0]:np.where(data.columns=='제직')[0][0]]),'위사_'))
list_col1.extend(data.columns[np.where(data.columns=='제직')[0][0]:])
data.columns=list_col1
np.array(data.columns)
#제직이후, Pilot전처리 이전까지 컬럼명 변경
list_col1=list(data.columns[np.where(data.columns=='번호')[0][0]:np.where(data.columns=='제직')[0][0]])
list_col1.extend(paste_python(list(data.loc[0][np.where(data.columns=='제직')[0][0]:np.where(data.columns=='Pilot전처리')[0][0]]),'제직_'))
list_col1.extend(data.columns[np.where(data.columns=='Pilot전처리')[0][0]:])
data.columns=list_col1
np.array(data.columns)
#Pilot전처리 이후, Pilot 후가공 공정 후 물성까지 컬럼명 변경
list_col1=list(data.columns[np.where(data.columns=='번호')[0][0]:np.where(data.columns=='Pilot전처리')[0][0]])
list_col1.extend(list(data.loc[0][np.where(data.columns=='Pilot전처리')[0][0]:np.where(data.columns=='Pilot 후가공 공정 후 물성')[0][0]]))
list_col1.extend(data.columns[np.where(data.columns=='Pilot 후가공 공정 후 물성')[0][0]:])
data.columns=list_col1
np.array(data.columns)
#검사물성 컬럼명 변경
list_col1=list(data.columns[np.where(data.columns=='번호')[0][0]:np.where(data.columns=='Pilot 후가공 공정 후 물성')[0][0]])

list_col1.extend(paste_python(list(data.loc[0][np.where(data.columns=='Pilot 후가공 공정 후 물성')[0][0]:np.where(data.columns=='Pilot 전처리 공정 후 물성')[0][0]]),'Pilot 후가공 공정 후 물성_'))
list_col1.extend(paste_python(list(data.loc[0][np.where(data.columns=='Pilot 전처리 공정 후 물성')[0][0]:np.where(data.columns=='Lab 후처리 환원세정 전 검사')[0][0]]),'Pilot 전처리 공정 후 물성_'))
list_col1.extend(paste_python(list(data.loc[0][np.where(data.columns=='Lab 후처리 환원세정 전 검사')[0][0]:np.where(data.columns=='Lab 후처리 환원세정 후 검사')[0][0]]),'Lab 후처리 환원세정 전 검사_'))
list_col1.extend(paste_python(list(data.loc[0][np.where(data.columns=='Lab 후처리 환원세정 후 검사')[0][0]:np.where(data.columns=='잔욕염색 검사')[0][0]]),'Lab 후처리 환원세정 후 검사_'))
list_col1.extend(paste_python(list(data.loc[0][np.where(data.columns=='잔욕염색 검사')[0][0]:np.where(data.columns=='Lab 후가공 후 검사')[0][0]]),'잔욕염색 검사_'))
list_col1.extend(paste_python(list(data.loc[0][np.where(data.columns=='Lab 후가공 후 검사')[0][0]:np.where(data.columns=='내광성 시험 전 검사')[0][0]]),'Lab 후가공 후 검사_'))
list_col1.extend(paste_python(list(data.loc[0][np.where(data.columns=='내광성 시험 전 검사')[0][0]:np.where(data.columns=='내광성 시험 후 검사')[0][0]]),'내광성 시험 전 검사_'))
list_col1.extend(paste_python(list(data.loc[0][np.where(data.columns=='내광성 시험 후 검사')[0][0]:(np.where(data.columns=='내광성 시험 후 검사')[0][0]+6)]),'내광성 시험 후 검사_'))

data.columns=list_col1
np.array(data.columns)
list_col1

['번호',
 '시료번호',
 '내광성시험분석',
 '접수번호No.',
 '업체명',
 '염색법',
 '기계분류ID',
 '기계분류',
 '원료ID',
 '경사_소재명',
 '경사_섬도',
 '경사_필라멘트수',
 '경사_광택',
 '경사_특성',
 '경사_밀도',
 '경사_성상',
 '경사_형상',
 '위사_소재명',
 '위사_섬도',
 '위사_필라멘트수',
 '위사_광택',
 '위사_특성',
 '위사_밀도',
 '위사_성상',
 '위사_형상',
 '제직_원단구조',
 '제직_조직',
 '제직_중량_생지',
 '제직_폭_생지',
 '제직_수량',
 'Pilot 전처리 기계장비ID',
 'Pilot 전처리 기계장비명',
 'Pilot 전처리 원단 투입량',
 'Pilot 전처리 조제 #1 제품명',
 'Pilot 전처리 조제 #1제조사명',
 'Pilot 전처리 조제 #1 종류',
 'Pilot 전처리 조제 #1투입량',
 'Pilot 전처리 조제 #2 제품명',
 'Pilot 전처리 조제 #2제조사명',
 'Pilot 전처리 조제 #2 종류',
 'Pilot 전처리 조제 #2투입량',
 'Pilot 전처리 조제 #3 제품명',
 'Pilot 전처리 조제 #3제조사명',
 'Pilot 전처리 조제 #3 종류',
 'Pilot 전처리 조제 #3투입량',
 'Pilot 전처리 시작온도',
 'Pilot 전처리 상승속도 #1',
 'Pilot 전처리 상승온도 #1',
 'Pilot 전처리 상승온도 #1 유지시간',
 'Pilot 전처리 상승속도 #2',
 'Pilot 전처리 상승온도 #2',
 'Pilot 전처리 상승온도 #2 유지시간',
 'Pilot 전처리 상승속도 #3',
 'Pilot 전처리 상승온도 #3',
 'Pilot 전처리 상승온도 #3 유지시간',
 'Pilot 전처리 하강속도 #1',
 'Pilot 전처리 하강온도 #1',
 'Pilot 전처리 하강온도 #1 유지시간',
 'Pilot 전처리 하강속도 #2',
 'Pilot 전처리 하강온도 #2',

In [6]:
data=data.drop([0],axis=0)
data=data.reset_index()
data=data[data.columns[1:]]
data

,번호,시료번호,내광성시험분석,접수번호No.,업체명,염색법,기계분류ID,기계분류,원료ID,경사_소재명,...,Lab 후가공 후 검사_건식마찰착색성,내광성 시험 전 검사_L*,내광성 시험 전 검사_a*,내광성 시험 전 검사_b*,내광성 시험 후 검사_L*,내광성 시험 후 검사_a*,내광성 시험 후 검사_b*,내광성 시험 후 검사_내광성등급,내광성 시험 후 검사_색차,내광성 시험 후 검사_비고
0,1273,209,카트리(2차),2021-10-18 00:00:00,다이텍연구원,1,P-DY09,고압 래피드 염색기,RAW-029,PET 재생 섬유,...,4,47.99,60.25,20.79,52.926667,49.953333,16.43,1,12.22,NaN
1,1274,209,카트리(2차),2021-10-18 00:00:00,다이텍연구원,1,P-DY09,고압 래피드 염색기,RAW-029,PET 재생 섬유,...,4,47.99,60.25,20.79,52.926667,49.953333,16.43,1,12.22,NaN
2,1275,210,카트리(2차),2021-10-18 00:00:00,다이텍연구원,1,P-DY09,고압 래피드 염색기,RAW-029,PET 재생 섬유,...,4,46.64,61.99,23.04,49.993333,52.37,17.73,1,11.49,NaN
3,1276,210,카트리(2차),2021-10-18 00:00:00,다이텍연구원,1,P-DY09,고압 래피드 염색기,RAW-029,PET 재생 섬유,...,4,46.64,61.99,23.04,49.993333,52.37,17.73,1,11.49,NaN
4,1277,211,카트리(2차),2021-10-18 00:00:00,다이텍연구원,1,P-DY09,고압 래피드 염색기,RAW-029,PET 재생 섬유,...,4,45.43,61.96,23.73,48.826667,51.5,17.813333,1,12.49,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3165,5602,798,1차년도_다이텍,2022-11-15 00:00:00,다이텍연구원,1,P-DY09,고압 래피드 염색기,RAW-029,PET 재생 섬유,...,4,25.73,9.35,-23.05,31.196667,7.793333,-15.956667,1.5,9.09,NaN
3166,5603,799,1차년도_다이텍,2022-11-15 00:00:00,다이텍연구원,1,P-DY09,고압 래피드 염색기,RAW-029,PET 재생 섬유,...,4,25.72,9.16,-22.72,30.626667,7.673333,-16.13,1.5,8.36,NaN
3167,5604,799,1차년도_다이텍,2022-11-15 00:00:00,다이텍연구원,1,P-DY09,고압 래피드 염색기,RAW-029,PET 재생 섬유,...,4,25.72,9.16,-22.72,30.626667,7.673333,-16.13,1.5,8.36,NaN
3168,5605,800,1차년도_다이텍,2022-11-15 00:00:00,다이텍연구원,1,P-DY09,고압 래피드 염색기,RAW-029,PET 재생 섬유,...,4,25.73,9.22,-22.65,30.413333,7.65,-16.283333,1.5,8.06,NaN


In [7]:
data.to_csv('C:/Users/leewa/Desktop/상위 컬럼명 변경.csv',encoding='euc-kr')

In [8]:
np.array(data.columns)

array(['번호', '시료번호', '내광성시험분석', '접수번호No.', '업체명', '염색법', '기계분류ID', '기계분류',
       '원료ID', '경사_소재명', '경사_섬도', '경사_필라멘트수', '경사_광택', '경사_특성', '경사_밀도',
       '경사_성상', '경사_형상', '위사_소재명', '위사_섬도', '위사_필라멘트수', '위사_광택', '위사_특성',
       '위사_밀도', '위사_성상', '위사_형상', '제직_원단구조', '제직_조직', '제직_중량_생지',
       '제직_폭_생지', '제직_수량', 'Pilot 전처리 기계장비ID', 'Pilot 전처리 기계장비명',
       'Pilot 전처리 원단 투입량', 'Pilot 전처리 조제 #1 제품명', 'Pilot 전처리 조제 #1제조사명',
       'Pilot 전처리 조제 #1 종류', 'Pilot 전처리 조제 #1투입량', 'Pilot 전처리 조제 #2 제품명',
       'Pilot 전처리 조제 #2제조사명', 'Pilot 전처리 조제 #2 종류', 'Pilot 전처리 조제 #2투입량',
       'Pilot 전처리 조제 #3 제품명', 'Pilot 전처리 조제 #3제조사명', 'Pilot 전처리 조제 #3 종류',
       'Pilot 전처리 조제 #3투입량', 'Pilot 전처리 시작온도', 'Pilot 전처리 상승속도 #1',
       'Pilot 전처리 상승온도 #1', 'Pilot 전처리 상승온도 #1 유지시간', 'Pilot 전처리 상승속도 #2',
       'Pilot 전처리 상승온도 #2', 'Pilot 전처리 상승온도 #2 유지시간', 'Pilot 전처리 상승속도 #3',
       'Pilot 전처리 상승온도 #3', 'Pilot 전처리 상승온도 #3 유지시간', 'Pilot 전처리 하강속도 #1',
       'Pilot 전처리 하강온도 #1', 'Pilot 전처리 하강온도 #1 유지시간', 'Pi

In [9]:
#필요없는 변수 제거
data=data.drop(['번호','시료번호','내광성시험분석','접수번호No.','업체명','염색법','기계분류ID','기계분류','원료ID','제직_수량','Pilot 전처리 기계장비ID',
                 'Pilot 전처리 건조장비ID','Lab 염색 기계 ID','Lab 후처리 기계ID','Lab 후가공 패더장비 ID','Lab 후가공 텐터장비ID',
                  '내광성 시험 전 검사_L*', '내광성 시험 전 검사_a*', '내광성 시험 전 검사_b*',
                   '내광성 시험 후 검사_L*', '내광성 시험 후 검사_a*', '내광성 시험 후 검사_b*','내광성 시험 후 검사_색차',
                'Lab 후가공 후 검사_중량', 'Lab 후가공 후 검사_경사_밀도', 'Lab 후가공 후 검사_위사_밀도',
                 'Lab 후가공 후 검사_K/S', '내광성 시험 후 검사_내광성등급', '내광성 시험 후 검사_비고',
               'Pilot 전처리 공정 후 물성_중량', 'Pilot 전처리 공정 후 물성_경사_밀도',
               'Pilot 전처리 공정 후 물성_위사_밀도', 'Pilot 전처리 공정 후 물성_원단 폭',
#Pilot 변수들은 아직 사용하지 않음.             
       'Lab 후가공 pick-up율', 'Pilot 배합 원단 중량',
       'Pilot 배합 염료 #1 명', 'Pilot 배합 염료 #1 제조사명', 'Pilot 배합 염료 #1 종류',
       'Pilot 배합 염료 #1 타입', 'Pilot 배합 염료 #1 투입량', 'Pilot 배합 염료 #2 명',
       'Pilot 배합 염료 #2 제조사명', 'Pilot 배합 염료 #2 종류', 'Pilot 배합 염료 #2 타입',
       'Pilot 배합 염료 #2 투입량', 'Pilot 배합 염료 #3 명', 'Pilot 배합 염료 #3 제조사명',
       'Pilot 배합 염료 #3 종류', 'Pilot 배합 염료 #3 타입', 'Pilot 배합 염료 #3 투입량',
       'Pilot 배합 염료 #4 명', 'Pilot 배합 염료 #4 제조사명', 'Pilot 배합 염료 #4 종류',
       'Pilot 배합 염료 #4 타입', 'Pilot 배합 염료 #4 투입량', 'Pilot 배합 액비',
       'Pilot 배합 용수량', 'Pilot 배합 조제 #1 명', 'Pilot 배합 조제 #1 제조사명',
       'Pilot 배합 조제 #1 종류', 'Pilot 배합 조제 #1 투입량', 'Pilot 배합 조제 #2 명',
       'Pilot 배합 조제 #2 제조사명', 'Pilot 배합 조제 #2 종류', 'Pilot 배합 조제 #2 투입량',
       'Pilot 배합 조제 #3 명', 'Pilot 배합 조제 #3 제조사명', 'Pilot 배합 조제 #3 종류',
       'Pilot 배합 조제 #3 투입량', 'Pilot 배합 조제 #4 명', 'Pilot 배합 조제 #4 제조사명',
       'Pilot 배합 조제 #4 종류', 'Pilot 배합 조제 #4 투입량', 'Pilot 염색 장비ID',
       'Pilot 염색 장비명', 'Pilot 염색 시작온도', 'Pilot 염색 상승속도 #1',
       'Pilot 염색 상승온도 #1', 'Pilot 염색 상승온도 #1 유지시간', 'Pilot 염색 상승속도 #2',
       'Pilot 염색 상승온도 #2', 'Pilot 염색 상승온도 #2 유지시간', 'Pilot 염색 상승속도 #3',
       'Pilot 염색 상승온도 #3', 'Pilot 염색 상승온도 #3 유지시간', 'Pilot 염색 하강속도 #1',
       'Pilot 염색 하강온도 #1', 'Pilot 염색 하강온도 #1 유지시간', 'Pilot 염색 하강속도 #2',
       'Pilot 염색 하강온도 #2', 'Pilot 염색 하강온도 #2 유지시간', 'Pilot 염색 하강속도 #3',
       'Pilot 염색 하강온도 #3', 'Pilot 염색 하강온도 #3 유지시간', 'Pilot 염색 종료속도',
       'Pilot 염색 종료온도', 'Pilot 염색 종료온도 유지시간', 'Pilot 염색 펌프압력',
       'Pilot 염색 릴속도', 'Pilot 염색 재염횟수', 'Pilot 후처리 장비ID', 'Pilot 후처리 장비명',
       'Pilot 후처리 조제 #1 명', 'Pilot 후처리 조제 #1 제조사명', 'Pilot 후처리 조제 #1 종류',
       'Pilot 후처리 조제 #1 투입량', 'Pilot 후처리 조제 #2 명', 'Pilot 후처리 조제 #2 제조사명',
       'Pilot 후처리 조제 #2 종류', 'Pilot 후처리 조제 #2 투입량', 'Pilot 후처리 환원세정 액비',
       'Pilot 후처리 환원세정 용수량', 'Pilot 후처리 환원세정 시작온도',
       'Pilot 후처리 환원세정 상승속도 #1', 'Pilot 후처리 환원세정 상승온도 #1',
       'Pilot 후처리 환원세정 상승온도 #1 유지시간', 'Pilot 후처리 환원세정 상승속도 #2',
       'Pilot 후처리 환원세정 상승온도 #2', 'Pilot 후처리 환원세정 상승온도 #2 유지시간',
       'Pilot 후처리 환원세정 상승속도 #3', 'Pilot 후처리 환원세정 상승온도 #3',
       'Pilot 후처리 환원세정 상승온도 #3 유지시간', 'Pilot 후처리 환원세정 하강속도 #1',
       'Pilot 후처리 환원세정 하강온도 #1', 'Pilot 후처리 환원세정 하강온도 #1 유지시간',
       'Pilot 후처리 환원세정 하강속도 #2', 'Pilot 후처리 환원세정 하강온도 #2',
       'Pilot 후처리 환원세정 하강온도 #2 유지시간', 'Pilot 후처리 환원세정 하강속도 #3',
       'Pilot 후처리 환원세정 하강온도 #3', 'Pilot 후처리 환원세정 하강온도 #3 유지시간',
       'Pilot 후처리 환원세정 종료속도', 'Pilot 후처리 환원세정 종료온도',
       'Pilot 후처리 환원세정 종료온도 유지시간', 'Pilot 후처리 수세온도 #1',
       'Pilot 후처리 수세시간 #1', 'Pilot 후처리 수세횟수 #1', 'Pilot 후처리 수세온도 #2',
       'Pilot 후처리 수세시간 #2', 'Pilot 후처리 수세횟수 #2', 'Pilot 후처리 수세온도 #3',
       'Pilot 후처리 수세시간 #3', 'Pilot 후처리 수세횟수 #3', 'Pilot 후가공 장비ID',
       'Pilot 후가공 장비명', 'Pilot 후가공 원단 투입량', 'Pilot 후가공 조제 명',
       'Pilot 후가공 조제 제조사명', 'Pilot 후가공 조제 종류', 'Pilot 후가공 조제 투입량',
       'Pilot 후가공 오버피드', 'Pilot 후가공 pick-up율', 'Pilot 후가공 오른쪽 패더압력',
       'Pilot 후가공 왼쪽 패더압력', 'Pilot 후가공 오른쪽 실제포온도 #1',
       'Pilot 후가공 왼쪽 실제포온도 #1', 'Pilot 후가공 오른쪽 세팅챔버온도 #1',
       'Pilot 후가공 왼쪽 세팅챔버온도 #1', 'Pilot 후가공 건조속도 #1', 'Pilot 후가공 건조열풍 #1',
       'Pilot 후가공 오른쪽 실제포온도 #2', 'Pilot 후가공 왼쪽 실제포온도 #2',
       'Pilot 후가공 오른쪽 세팅챔버온도 #2', 'Pilot 후가공 왼쪽 세팅챔버온도 #2',
       'Pilot 후가공 건조속도 #2', 'Pilot 후가공 건조열풍 #2', 'Pilot 후가공 오른쪽 실제포온도 #3',
       'Pilot 후가공 왼쪽 실제포온도 #3', 'Pilot 후가공 오른쪽 세팅챔버온도 #3',
       'Pilot 후가공 왼쪽 세팅챔버온도 #3', 'Pilot 후가공 건조속도 #3', 'Pilot 후가공 건조열풍 #3',
       'Pilot 후가공 오른쪽 실제포온도 #4', 'Pilot 후가공 왼쪽 실제포온도 #4',
       'Pilot 후가공 오른쪽 세팅챔버온도 #4', 'Pilot 후가공 왼쪽 세팅챔버온도 #4',
       'Pilot 후가공 건조속도 #4', 'Pilot 후가공 건조열풍 #4', 'Pilot 후가공 오른쪽 실제포온도 #5',
       'Pilot 후가공 왼쪽 실제포온도 #5', 'Pilot 후가공 오른쪽 세팅챔버온도 #5',
       'Pilot 후가공 왼쪽 세팅챔버온도 #5', 'Pilot 후가공 건조속도 #5', 'Pilot 후가공 건조열풍 #5',
       'Pilot 후가공 오른쪽 실제포온도 #6', 'Pilot 후가공 왼쪽 실제포온도 #6',
       'Pilot 후가공 오른쪽 세팅챔버온도 #6', 'Pilot 후가공 왼쪽 세팅챔버온도 #6',
       'Pilot 후가공 건조속도 #6', 'Pilot 후가공 건조열풍 #6', 'Pilot 후가공 오른쪽 실제포온도 #7',
       'Pilot 후가공 왼쪽 실제포온도 #7', 'Pilot 후가공 오른쪽 세팅챔버온도 #7',
       'Pilot 후가공 왼쪽 세팅챔버온도 #7', 'Pilot 후가공 건조속도 #7', 'Pilot 후가공 건조열풍 #7',
       'Pilot 후가공 공정 후 물성_중량', 'Pilot 후가공 공정 후 물성_원단 폭',
#검사쪽은 공정 이후에 발생가능하므로 lab후가공 후 lab 및 k/s를 제외하고 ai모델개발 대상에서 제외
       'Lab 후처리 환원세정 전 검사_L*', 'Lab 후처리 환원세정 전 검사_a*',
       'Lab 후처리 환원세정 전 검사_b*', 'Lab 후처리 환원세정 전 검사_K/S',
       'Lab 후처리 환원세정 후 검사_L*', 'Lab 후처리 환원세정 후 검사_a*',
       'Lab 후처리 환원세정 후 검사_b*', 'Lab 후처리 환원세정 후 검사_K/S'
               ],axis=1)
data

,경사_소재명,경사_섬도,경사_필라멘트수,경사_광택,경사_특성,경사_밀도,경사_성상,경사_형상,위사_소재명,위사_섬도,...,잔욕염색 검사_b*,잔욕염색 검사_K/S,Lab 후가공 후 검사_L*,Lab 후가공 후 검사_a*,Lab 후가공 후 검사_b*,Lab 후가공 후 검사_내마모성,Lab 후가공 후 검사_세탁견뢰도_시료,Lab 후가공 후 검사_세탁견뢰도_백면포,Lab 후가공 후 검사_습식마찰착색성,Lab 후가공 후 검사_건식마찰착색성
0,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,150*2 INT,Filaments,PET 재생 섬유,300,...,15.55,13.4,46.93,62.04,20.34,4,4.5,4,4,4
1,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,150*2 INT,Filaments,PET 재생 섬유,300,...,15.77,13.9,47.19,62.13,20.55,4,4.5,4,4,4
2,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,150*2 INT,Filaments,PET 재생 섬유,300,...,13.65,11.1,45.45,62.54,21.56,4,4.5,4,4.5,4
3,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,150*2 INT,Filaments,PET 재생 섬유,300,...,14.05,11.8,45.55,62.43,21.49,4,4.5,4,4.5,4
4,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,150*2 INT,Filaments,PET 재생 섬유,300,...,11.83,8.872,44.31,62.57,22.47,4,4.5,4,4.5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3165,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,150*2 INT,Filaments,PET 재생 섬유,300,...,-5.3225,0.33075,25.0575,9.055,-22.8125,3,4,4,4.5,4
3166,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,150*2 INT,Filaments,PET 재생 섬유,300,...,-4.5425,0.3095,25.035,9.0425,-22.72,3,4,4,4.5,4
3167,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,150*2 INT,Filaments,PET 재생 섬유,300,...,-4.915,0.32575,25.185,9.045,-22.7475,3,4,4,4.5,4
3168,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,150*2 INT,Filaments,PET 재생 섬유,300,...,-4.75,0.3095,25.2225,9.045,-22.5875,3,4,4,4.5,4


# 데이터 전처리(Pivot, Prop화)

In [10]:
전처리=data[[#'Pilot 전처리 기계장비명',
       'Pilot 전처리 원단 투입량', 'Pilot 전처리 조제 #1 제품명',
       'Pilot 전처리 조제 #1제조사명', 'Pilot 전처리 조제 #1 종류', 'Pilot 전처리 조제 #1투입량',
       'Pilot 전처리 조제 #2 제품명', 'Pilot 전처리 조제 #2제조사명', 'Pilot 전처리 조제 #2 종류',
       'Pilot 전처리 조제 #2투입량', 'Pilot 전처리 조제 #3 제품명', 'Pilot 전처리 조제 #3제조사명',
       'Pilot 전처리 조제 #3 종류', 'Pilot 전처리 조제 #3투입량', 'Pilot 전처리 시작온도',
       'Pilot 전처리 상승속도 #1', 'Pilot 전처리 상승온도 #1', 'Pilot 전처리 상승온도 #1 유지시간',
       'Pilot 전처리 상승속도 #2', 'Pilot 전처리 상승온도 #2', 'Pilot 전처리 상승온도 #2 유지시간',
       'Pilot 전처리 상승속도 #3', 'Pilot 전처리 상승온도 #3', 'Pilot 전처리 상승온도 #3 유지시간',
       'Pilot 전처리 하강속도 #1', 'Pilot 전처리 하강온도 #1', 'Pilot 전처리 하강온도 #1 유지시간',
       'Pilot 전처리 하강속도 #2', 'Pilot 전처리 하강온도 #2', 'Pilot 전처리 하강온도 #2 유지시간',
       'Pilot 전처리 하강속도 #3', 'Pilot 전처리 하강온도 #3', 'Pilot 전처리 하강온도 #3 유지시간',
       'Pilot 전처리 종료속도', 'Pilot 전처리 종료온도', 'Pilot 전처리 종료온도 유지시간',
       'Pilot 전처리 수세온도', 'Pilot 전처리 수세시간', 'Pilot 전처리 수세횟수',
       'Pilot 전처리 건조장비', 'Pilot 전처리 오버피드', 'Pilot 전처리 오른쪽 패더압력',
       'Pilot 전처리 왼쪽 패더압력', 'Pilot 전처리 오른쪽 실제포온도 #1',
       'Pilot 전처리 왼쪽 실제포온도 #1', 'Pilot 전처리 오른쪽 세팅챔버온도 #1',
       'Pilot 전처리 왼쪽 세팅챔버온도 #1', 'Pilot 전처리 건조속도 #1', 'Pilot 전처리 건조열풍 #1',
       'Pilot 전처리 오른쪽 실제포온도 #2', 'Pilot 전처리 왼쪽 실제포온도 #2',
       'Pilot 전처리 오른쪽 세팅챔버온도 #2', 'Pilot 전처리 왼쪽 세팅챔버온도 #2',
       'Pilot 전처리 건조속도 #2', 'Pilot 전처리 건조열풍 #2', 'Pilot 전처리 오른쪽 실제포온도 #3',
       'Pilot 전처리 왼쪽 실제포온도 #3', 'Pilot 전처리 오른쪽 세팅챔버온도 #3',
       'Pilot 전처리 왼쪽 세팅챔버온도 #3', 'Pilot 전처리 건조속도 #3', 'Pilot 전처리 건조열풍 #3',
       'Pilot 전처리 오른쪽 실제포온도 #4', 'Pilot 전처리 왼쪽 실제포온도 #4',
       'Pilot 전처리 오른쪽 세팅챔버온도 #4', 'Pilot 전처리 왼쪽 세팅챔버온도 #4',
       'Pilot 전처리 건조속도 #4', 'Pilot 전처리 건조열풍 #4', 'Pilot 전처리 오른쪽 실제포온도 #5',
       'Pilot 전처리 왼쪽 실제포온도 #5', 'Pilot 전처리 오른쪽 세팅챔버온도 #5',
       'Pilot 전처리 왼쪽 세팅챔버온도 #5', 'Pilot 전처리 건조속도 #5', 'Pilot 전처리 건조열풍 #5',
       'Pilot 전처리 오른쪽 실제포온도 #6', 'Pilot 전처리 왼쪽 실제포온도 #6',
       'Pilot 전처리 오른쪽 세팅챔버온도 #6', 'Pilot 전처리 왼쪽 세팅챔버온도 #6',
       'Pilot 전처리 건조속도 #6', 'Pilot 전처리 건조열풍 #6', 'Pilot 전처리 오른쪽 실제포온도 #7',
       'Pilot 전처리 왼쪽 실제포온도 #7', 'Pilot 전처리 오른쪽 세팅챔버온도 #7',
       'Pilot 전처리 왼쪽 세팅챔버온도 #7', 'Pilot 전처리 건조속도 #7', 'Pilot 전처리 건조열풍 #7']]

In [11]:
len(전처리.columns)

84

In [12]:
전처리_조제=전처리[['Pilot 전처리 조제 #1 제품명','Pilot 전처리 조제 #1투입량',
            'Pilot 전처리 조제 #2 제품명','Pilot 전처리 조제 #2투입량',
           'Pilot 전처리 조제 #3 제품명','Pilot 전처리 조제 #3투입량']]


In [13]:
전처리_조제

,Pilot 전처리 조제 #1 제품명,Pilot 전처리 조제 #1투입량,Pilot 전처리 조제 #2 제품명,Pilot 전처리 조제 #2투입량,Pilot 전처리 조제 #3 제품명,Pilot 전처리 조제 #3투입량
0,Sera Wash C-NEP,3,NaOH,5,CH3COOH,1
1,Sera Wash C-NEP,3,NaOH,5,CH3COOH,1
2,Sera Wash C-NEP,3,NaOH,5,CH3COOH,1
3,Sera Wash C-NEP,3,NaOH,5,CH3COOH,1
4,Sera Wash C-NEP,3,NaOH,5,CH3COOH,1
...,...,...,...,...,...,...
3165,Sera Wash C-NEP,3,NaOH,5,CH3COOH,1
3166,Sera Wash C-NEP,3,NaOH,5,CH3COOH,1
3167,Sera Wash C-NEP,3,NaOH,5,CH3COOH,1
3168,Sera Wash C-NEP,3,NaOH,5,CH3COOH,1


In [14]:
전처리_조제_df=pivot_PET(전처리_조제)
전처리_조제_df.columns='전처리_'+전처리_조제_df.columns
전처리_조제_df


,전처리_Sera Wash C-NEP,전처리_NaOH,전처리_CH3COOH
0,3,5,1
1,3,5,1
2,3,5,1
3,3,5,1
4,3,5,1
...,...,...,...
3165,3,5,1
3166,3,5,1
3167,3,5,1
3168,3,5,1


In [15]:
전처리_온도_df=data[['Pilot 전처리 시작온도',
       'Pilot 전처리 상승속도 #1', 'Pilot 전처리 상승온도 #1', 'Pilot 전처리 상승온도 #1 유지시간',
       'Pilot 전처리 상승속도 #2', 'Pilot 전처리 상승온도 #2', 'Pilot 전처리 상승온도 #2 유지시간',
       'Pilot 전처리 상승속도 #3', 'Pilot 전처리 상승온도 #3', 'Pilot 전처리 상승온도 #3 유지시간',
       'Pilot 전처리 하강속도 #1', 'Pilot 전처리 하강온도 #1', 'Pilot 전처리 하강온도 #1 유지시간',
       'Pilot 전처리 하강속도 #2', 'Pilot 전처리 하강온도 #2', 'Pilot 전처리 하강온도 #2 유지시간',
       'Pilot 전처리 하강속도 #3', 'Pilot 전처리 하강온도 #3', 'Pilot 전처리 하강온도 #3 유지시간',
       'Pilot 전처리 종료속도', 'Pilot 전처리 종료온도', 'Pilot 전처리 종료온도 유지시간',
       'Pilot 전처리 수세온도', 'Pilot 전처리 수세시간', 'Pilot 전처리 수세횟수'
               ]]
#세팅챔버온도 및 건조속도, 건조풍속 평균냄


#실제포온도 삭제
전처리_건조_df=data[[
       'Pilot 전처리 오버피드', 
       'Pilot 전처리 오른쪽 패더압력','Pilot 전처리 왼쪽 패더압력', 
       'Pilot 전처리 오른쪽 세팅챔버온도 #1', 'Pilot 전처리 왼쪽 세팅챔버온도 #1', 'Pilot 전처리 건조속도 #1', 'Pilot 전처리 건조열풍 #1',
       'Pilot 전처리 오른쪽 세팅챔버온도 #2', 'Pilot 전처리 왼쪽 세팅챔버온도 #2', 'Pilot 전처리 건조속도 #2', 'Pilot 전처리 건조열풍 #2',
       'Pilot 전처리 오른쪽 세팅챔버온도 #3', 'Pilot 전처리 왼쪽 세팅챔버온도 #3', 'Pilot 전처리 건조속도 #3', 'Pilot 전처리 건조열풍 #3',
       'Pilot 전처리 오른쪽 세팅챔버온도 #4', 'Pilot 전처리 왼쪽 세팅챔버온도 #4', 'Pilot 전처리 건조속도 #4', 'Pilot 전처리 건조열풍 #4', 
       'Pilot 전처리 오른쪽 세팅챔버온도 #5', 'Pilot 전처리 왼쪽 세팅챔버온도 #5', 'Pilot 전처리 건조속도 #5', 'Pilot 전처리 건조열풍 #5',
       'Pilot 전처리 오른쪽 세팅챔버온도 #6', 'Pilot 전처리 왼쪽 세팅챔버온도 #6', 'Pilot 전처리 건조속도 #6', 'Pilot 전처리 건조열풍 #6',
       'Pilot 전처리 오른쪽 세팅챔버온도 #7', 'Pilot 전처리 왼쪽 세팅챔버온도 #7', 'Pilot 전처리 건조속도 #7', 'Pilot 전처리 건조열풍 #7'
]]

#세팅챔버온도 및 건조속도, 건조풍속 평균냄
전처리_건조_df['Pilot 전처리 세팅챔버온도']=전처리_건조_df.apply(lambda x : np.mean(x[['Pilot 전처리 오른쪽 세팅챔버온도 #1',
                                                                                        'Pilot 전처리 오른쪽 세팅챔버온도 #2',
                                                                                        'Pilot 전처리 오른쪽 세팅챔버온도 #3',
                                                                                        'Pilot 전처리 오른쪽 세팅챔버온도 #4',
                                                                                        'Pilot 전처리 오른쪽 세팅챔버온도 #5',
                                                                                        'Pilot 전처리 오른쪽 세팅챔버온도 #6',
                                                                                        'Pilot 전처리 오른쪽 세팅챔버온도 #7',
                                                                                        'Pilot 전처리 왼쪽 세팅챔버온도 #1',
                                                                                        'Pilot 전처리 왼쪽 세팅챔버온도 #2',
                                                                                        'Pilot 전처리 왼쪽 세팅챔버온도 #3',
                                                                                        'Pilot 전처리 왼쪽 세팅챔버온도 #4',
                                                                                        'Pilot 전처리 왼쪽 세팅챔버온도 #5',
                                                                                        'Pilot 전처리 왼쪽 세팅챔버온도 #6',
                                                                                        'Pilot 전처리 왼쪽 세팅챔버온도 #7'
                                                                   ]]),axis=1)

전처리_건조_df['Pilot 전처리 건조속도']=전처리_건조_df.apply(lambda x : np.mean(x[[    'Pilot 전처리 건조속도 #1',
                                                                                        'Pilot 전처리 건조속도 #2',
                                                                                        'Pilot 전처리 건조속도 #3',
                                                                                        'Pilot 전처리 건조속도 #4',
                                                                                        'Pilot 전처리 건조속도 #5',
                                                                                        'Pilot 전처리 건조속도 #6',
                                                                                        'Pilot 전처리 건조속도 #7'
                                                                   ]]),axis=1)

전처리_건조_df['Pilot 전처리 건조열풍']=전처리_건조_df.apply(lambda x : np.mean(x[[    'Pilot 전처리 건조열풍 #1',
                                                                                        'Pilot 전처리 건조열풍 #2',
                                                                                        'Pilot 전처리 건조열풍 #3',
                                                                                        'Pilot 전처리 건조열풍 #4',
                                                                                        'Pilot 전처리 건조열풍 #5',
                                                                                        'Pilot 전처리 건조열풍 #6',
                                                                                        'Pilot 전처리 건조열풍 #7'
                                                                   ]]),axis=1)


전처리_건조_df=전처리_건조_df[[
       'Pilot 전처리 오버피드', 
       'Pilot 전처리 오른쪽 패더압력','Pilot 전처리 왼쪽 패더압력', 'Pilot 전처리 세팅챔버온도', 'Pilot 전처리 건조속도', 'Pilot 전처리 건조열풍'
                                ]]
#전처리_검사_df=data[['Pilot 전처리 공정 후 물성_중량',
#       'Pilot 전처리 공정 후 물성_원단 폭','Pilot 전처리 공정 후 물성_경사_밀도',
#       'Pilot 전처리 공정 후 물성_위사_밀도']]

전처리_건조_df
전처리=pd.concat([전처리_조제_df,전처리_온도_df,전처리_건조_df,
               #전처리_검사_df
              ],axis=1)
전처리

C:\Users\leewa\AppData\Local\Temp\ipykernel_6400\3013797395.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  전처리_건조_df['Pilot 전처리 세팅챔버온도']=전처리_건조_df.apply(lambda x : np.mean(x[['Pilot 전처리 오른쪽 세팅챔버온도 #1',
C:\Users\leewa\AppData\Local\Temp\ipykernel_6400\3013797395.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  전처리_건조_df['Pilot 전처리 건조속도']=전처리_건조_df.apply(lambda x : np.mean(x[[    'Pilot 전처리 건조속도 #1',
C:\Users\leewa\AppData\Local\Temp\ipykernel_6400\3013797395.py:53: SettingWithCopyWarning: 
A value 

,전처리_Sera Wash C-NEP,전처리_NaOH,전처리_CH3COOH,Pilot 전처리 시작온도,Pilot 전처리 상승속도 #1,Pilot 전처리 상승온도 #1,Pilot 전처리 상승온도 #1 유지시간,Pilot 전처리 상승속도 #2,Pilot 전처리 상승온도 #2,Pilot 전처리 상승온도 #2 유지시간,...,Pilot 전처리 종료온도 유지시간,Pilot 전처리 수세온도,Pilot 전처리 수세시간,Pilot 전처리 수세횟수,Pilot 전처리 오버피드,Pilot 전처리 오른쪽 패더압력,Pilot 전처리 왼쪽 패더압력,Pilot 전처리 세팅챔버온도,Pilot 전처리 건조속도,Pilot 전처리 건조열풍
0,3,5,1,45,5.5,95,60,0,95,0,...,0,80,10,2,1,3,3,170.0,20.0,1200.0
1,3,5,1,45,5.5,95,60,0,95,0,...,0,80,10,2,1,3,3,170.0,20.0,1200.0
2,3,5,1,45,5.5,95,60,0,95,0,...,0,80,10,2,1,3,3,170.0,20.0,1200.0
3,3,5,1,45,5.5,95,60,0,95,0,...,0,80,10,2,1,3,3,170.0,20.0,1200.0
4,3,5,1,45,5.5,95,60,0,95,0,...,0,80,10,2,1,3,3,170.0,20.0,1200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3165,3,5,1,45,5.5,95,60,0,95,0,...,0,80,10,2,1,3,3,170.0,20.0,1200.0
3166,3,5,1,45,5.5,95,60,0,95,0,...,0,80,10,2,1,3,3,170.0,20.0,1200.0
3167,3,5,1,45,5.5,95,60,0,95,0,...,0,80,10,2,1,3,3,170.0,20.0,1200.0
3168,3,5,1,45,5.5,95,60,0,95,0,...,0,80,10,2,1,3,3,170.0,20.0,1200.0


In [16]:
data['경사_성상_섬도 #1']=data['경사_성상'].str[0:3]
data['경사_성상_섬도 #2']=data['경사_성상'].str[0:3]
data['경사_성상_섬도 #3']=0
data['경사_성상_섬도 #4']=0

data['경사_성상_비율 #1']=50
data['경사_성상_비율 #2']=50
data['경사_성상_비율 #3']=0
data['경사_성상_비율 #4']=0

data['경사_성상_특성']='Interlace'

data['위사_성상_섬도 #1']=data['위사_성상'].str[0:3]
data['위사_성상_섬도 #2']=data['위사_성상'].str[0:3]
data['위사_성상_섬도 #3']=0
data['위사_성상_섬도 #4']=0

data['위사_성상_비율 #1']=50
data['위사_성상_비율 #2']=50
data['위사_성상_비율 #3']=0
data['위사_성상_비율 #4']=0

data['위사_성상_특성']='Interlace'

data=data.drop(['경사_성상','위사_성상'],axis=1)
data

소재=data[['경사_소재명', 
         '경사_섬도', 
         '경사_필라멘트수', 
         '경사_광택', 
         '경사_특성', 
         '경사_밀도',
         '경사_형상', 
         '경사_성상_섬도 #1',
         '경사_성상_섬도 #2','경사_성상_섬도 #3','경사_성상_섬도 #4',
        '경사_성상_비율 #1',
         '경사_성상_비율 #2','경사_성상_비율 #3','경사_성상_비율 #4',
        '경사_성상_특성',
         '위사_소재명', '위사_섬도', '위사_필라멘트수', '위사_광택', '위사_특성', '위사_밀도',
       '위사_형상', '위사_성상_섬도 #1','위사_성상_섬도 #2','위사_성상_섬도 #3','위사_성상_섬도 #4',
        '위사_성상_비율 #1','위사_성상_비율 #2','위사_성상_비율 #3','위사_성상_비율 #4',
         '위사_성상_특성',
        '제직_원단구조', '제직_조직','제직_중량_생지','제직_폭_생지']]
#소재=소재.fillna('none')
소재

,경사_소재명,경사_섬도,경사_필라멘트수,경사_광택,경사_특성,경사_밀도,경사_형상,경사_성상_섬도 #1,경사_성상_섬도 #2,경사_성상_섬도 #3,...,위사_성상_섬도 #4,위사_성상_비율 #1,위사_성상_비율 #2,위사_성상_비율 #3,위사_성상_비율 #4,위사_성상_특성,제직_원단구조,제직_조직,제직_중량_생지,제직_폭_생지
0,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0,50,50,0,0,Interlace,WOVEN,TWILL,237,1.37
1,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0,50,50,0,0,Interlace,WOVEN,TWILL,237,1.37
2,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0,50,50,0,0,Interlace,WOVEN,TWILL,237,1.37
3,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0,50,50,0,0,Interlace,WOVEN,TWILL,237,1.37
4,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0,50,50,0,0,Interlace,WOVEN,TWILL,237,1.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3165,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0,50,50,0,0,Interlace,WOVEN,TWILL,NaN,NaN
3166,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0,50,50,0,0,Interlace,WOVEN,TWILL,NaN,NaN
3167,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0,50,50,0,0,Interlace,WOVEN,TWILL,NaN,NaN
3168,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0,50,50,0,0,Interlace,WOVEN,TWILL,NaN,NaN


In [18]:
배합=data[[ 'Lab 배합 염료 #1 명','Lab 배합 염료 #1 투입량',
            'Lab 배합 염료 #2 명','Lab 배합 염료 #2 투입량',
            'Lab 배합 염료 #3 명','Lab 배합 염료 #3 투입량',
            'Lab 배합 염료 #4 명','Lab 배합 염료 #4 투입량',
                     ]]
배합_df=pivot_PET(배합)

#Dorosperse Brown K-3LR, Synofast 28가 없어서 추가해줘야함
#배합_df['Dorosperse Brown K-3LR']=np.nan
#배합_df['Synofast 28']=np.nan
배합_df=배합_df.fillna(0)
배합2=data[[ 'Lab 배합 조제 #1 명', 'Lab 배합 조제 #1 투입량',
             'Lab 배합 조제 #2 명', 'Lab 배합 조제 #2 투입량',
             'Lab 배합 조제 #3 명', 'Lab 배합 조제 #3 투입량',
             'Lab 배합 조제 #4 명', 'Lab 배합 조제 #4 투입량']]
배합2_df=pivot_PET(배합2)
배합2_df=배합2_df.fillna(0)
배합2_df.columns='배합_'+배합2_df.columns

배합=pd.concat([배합_df,배합2_df],axis=1)
배합

,Dorosperse Red KKL,Dorosperse Blue KKL,Dorosperse B/K KKL,Dorosperse Dark Grey KKL,Dorosperse Brown K-3LR,Dianix Yellow AM-2R,Dianix Red AM-SLR,Dianix Blue AM-2G,Dianix Black AM-SLR,Dianix Grey AM-SLR,Dianix Yellow Brown AM-R,Dorosperse Yellow KKL,Dorosperse Black KKL,배합_Sunsolt RM-340S,배합_빙초산
0,6.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3,0.2
1,6.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3,0.2
2,6.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3,0.2
3,6.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3,0.2
4,6.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3165,1.333,2.667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3,0.2
3166,1.333,2.667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3,0.2
3167,1.333,2.667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3,0.2
3168,1.333,2.667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3,0.2


In [19]:
염색=data[[#'Lab 염색 기계명', 
         'Lab 염색 시작온도',
       'Lab 염색 상승속도 #1', 'Lab 염색 상승온도 #1', 'Lab 염색 상승온도 #1 유지시간',
       'Lab 염색 상승속도 #2', 'Lab 염색 상승온도 #2', 'Lab 염색 상승온도 #2 유지시간',
       'Lab 염색 상승속도 #3', 'Lab 염색 상승온도 #3', 'Lab 염색 상승온도 #3 유지시간',
       'Lab 염색 하강속도 #1', 'Lab 염색 하강온도 #1', 'Lab 염색 하강온도 #1 유지시간',
       'Lab 염색 하강속도 #2', 'Lab 염색 하강온도 #2', 'Lab 염색 하강온도 #2 유지시간',
       'Lab 염색 하강속도 #3', 'Lab 염색 하강온도 #3', 'Lab 염색 하강온도 #3 유지시간',
       'Lab 염색 종료속도', 'Lab 염색 종료온도', 'Lab 염색 종료온도 유지시간']]
#후처리 조제 전처리
후처리=data[['Lab 후처리 조제 #1 명', 'Lab 후처리 조제 #1 투입량', 'Lab 후처리 조제 #2 명', 'Lab 후처리 조제 #2 투입량']]

후처리_df=pivot_PET(후처리)
후처리_df.columns='후처리_'+후처리_df.columns

# 3번째 수세변수 3개('Lab 후처리 수세온도 #3','Lab 후처리 수세시간 #3', 'Lab 후처리 수세횟수 #3') 제거
# 수세 시간변수 3개('Lab 후처리 수세시간 #1','Lab 후처리 수세시간 #2','Lab 후처리 수세시간 #3') 제거
후처리_df2=data[[#'Lab 후처리 기계명','Lab 후처리 환원세정 용수량', 
       'Lab 후처리 환원세정 시작온도', 'Lab 후처리 환원세정 상승속도 #1',
       'Lab 후처리 환원세정 상승온도 #1', 'Lab 후처리 환원세정 상승온도 #1 유지시간',
       'Lab 후처리 환원세정 상승속도 #2', 'Lab 후처리 환원세정 상승온도 #2',
       'Lab 후처리 환원세정 상승온도 #2 유지시간', 'Lab 후처리 환원세정 상승속도 #3',
       'Lab 후처리 환원세정 상승온도 #3', 'Lab 후처리 환원세정 상승온도 #3 유지시간',
       'Lab 후처리 환원세정 하강속도 #1', 'Lab 후처리 환원세정 하강온도 #1',
       'Lab 후처리 환원세정 하강온도 #1 유지시간', 'Lab 후처리 환원세정 하강속도 #2',
       'Lab 후처리 환원세정 하강온도 #2', 'Lab 후처리 환원세정 하강온도 #2 유지시간',
       'Lab 후처리 환원세정 하강속도 #3', 'Lab 후처리 환원세정 하강온도 #3',
       'Lab 후처리 환원세정 하강온도 #3 유지시간', 'Lab 후처리 환원세정 종료속도',
       'Lab 후처리 환원세정 종료온도', 'Lab 후처리 환원세정 종료온도 유지시간', 'Lab 후처리 수세온도 #1',
       'Lab 후처리 수세횟수 #1', 'Lab 후처리 수세온도 #2', 'Lab 후처리 수세횟수 #2', 'Lab 후처리 건조온도','Lab 후처리 건조시간']]
후처리_df2
후처리=pd.concat([후처리_df,후처리_df2],axis=1)

#현재 후가공시, 조제는 넣지 않으므로 우선, 변수 삭제
후가공=data[[#'Lab 후가공 패더장비명',
       'Lab 후가공 패더압력', 'Lab 후가공 패더속도', 
    #'Lab 후가공 텐터장비명', 
    'Lab 후가공 건조온도',
       'Lab 후가공 건조시간', 'Lab 후가공 열풍', 'Lab 후가공 스팀']]
후가공

,Lab 후가공 패더압력,Lab 후가공 패더속도,Lab 후가공 건조온도,Lab 후가공 건조시간,Lab 후가공 열풍,Lab 후가공 스팀
0,1,1,180,1,2,2
1,1,1,180,1,2,2
2,1,1,180,1,2,2
3,1,1,180,1,2,2
4,1,1,180,1,2,2
...,...,...,...,...,...,...
3165,1,1,180,1,2,2
3166,1,1,180,1,2,2
3167,1,1,180,1,2,2
3168,1,1,180,1,2,2


In [20]:
검사=data[['잔욕염색 검사_L*','잔욕염색 검사_a*', '잔욕염색 검사_b*', '잔욕염색 검사_K/S']]#, 'Lab 후가공 후 검사_K/S','내광성 시험 후 검사_내광성등급']]
검사

,잔욕염색 검사_L*,잔욕염색 검사_a*,잔욕염색 검사_b*,잔욕염색 검사_K/S
0,48.17,63.13,15.55,13.4
1,47.96,63.42,15.77,13.9
2,50.06,62.44,13.65,11.1
3,49.47,62.67,14.05,11.8
4,52,61.27,11.83,8.872
...,...,...,...,...
3165,74.5775,4.785,-5.3225,0.33075
3166,75.47,5.175,-4.5425,0.3095
3167,74.86,5.12,-4.915,0.32575
3168,75.3325,4.73,-4.75,0.3095


In [21]:
data_전처리=pd.concat([소재,전처리,배합,염색,후처리,후가공,검사],axis=1)
np.array(np.sum(data_전처리.isnull()))

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0, 406, 406,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0], dtype=int64)

In [22]:
data_전처리=data_전처리[data_전처리[['Dorosperse Red KKL', 'Dorosperse Blue KKL', 'Dorosperse B/K KKL',
       'Dorosperse Dark Grey KKL', 'Dorosperse Brown K-3LR',
       'Dianix Yellow AM-2R', 'Dianix Red AM-SLR', 'Dianix Blue AM-2G',
       'Dianix Black AM-SLR', 'Dianix Grey AM-SLR', 'Dianix Yellow Brown AM-R',
       'Dorosperse Yellow KKL', 'Dorosperse Black KKL']].sum(axis=1)!=0]

In [23]:
color_mat = data_전처리[['Dorosperse Red KKL', 'Dorosperse Blue KKL', 'Dorosperse B/K KKL',
       'Dorosperse Dark Grey KKL', 'Dorosperse Brown K-3LR',
       'Dianix Yellow AM-2R', 'Dianix Red AM-SLR', 'Dianix Blue AM-2G',
       'Dianix Black AM-SLR', 'Dianix Grey AM-SLR', 'Dianix Yellow Brown AM-R',
       'Dorosperse Yellow KKL', 'Dorosperse Black KKL']].values

In [24]:
col = ['Dorosperse Red KKL', 'Dorosperse Blue KKL', 'Dorosperse B/K KKL',
       'Dorosperse Dark Grey KKL', 'Dorosperse Brown K-3LR',
       'Dianix Yellow AM-2R', 'Dianix Red AM-SLR', 'Dianix Blue AM-2G',
       'Dianix Black AM-SLR', 'Dianix Grey AM-SLR', 'Dianix Yellow Brown AM-R',
       'Dorosperse Yellow KKL', 'Dorosperse Black KKL']

In [25]:
data_전처리[[i+"_prop" for i in col]] =color_mat/color_mat.sum(axis=1).reshape(-1,1)

In [26]:
prop=data_전처리[[ 'Dorosperse Red KKL_prop', 'Dorosperse Blue KKL_prop', 'Dorosperse B/K KKL_prop',
       'Dorosperse Dark Grey KKL_prop', 'Dorosperse Brown K-3LR_prop',
       'Dianix Yellow AM-2R_prop', 'Dianix Red AM-SLR_prop', 'Dianix Blue AM-2G_prop',
       'Dianix Black AM-SLR_prop', 'Dianix Grey AM-SLR_prop', 'Dianix Yellow Brown AM-R_prop',
       'Dorosperse Yellow KKL_prop', 'Dorosperse Black KKL_prop']]
prop.describe()


,Dorosperse Red KKL_prop,Dorosperse Blue KKL_prop,Dorosperse B/K KKL_prop,Dorosperse Dark Grey KKL_prop,Dorosperse Brown K-3LR_prop,Dianix Yellow AM-2R_prop,Dianix Red AM-SLR_prop,Dianix Blue AM-2G_prop,Dianix Black AM-SLR_prop,Dianix Grey AM-SLR_prop,Dianix Yellow Brown AM-R_prop,Dorosperse Yellow KKL_prop,Dorosperse Black KKL_prop
count,3170.000000,3170.000000,3170.000000,3170.000000,3170.000000,3170.000000,3170.000000,3170.000000,3170.000000,3170.000000,3170.000000,3170.000000,3170.000000
mean,0.205468,0.206940,0.025237,0.061199,0.061199,0.034069,0.034069,0.034069,0.034069,0.034069,0.034069,0.199579,0.035962
std,0.306961,0.308367,0.156868,0.239732,0.239732,0.181436,0.181436,0.181436,0.181436,0.181436,0.181436,0.302824,0.186225
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.333333,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [27]:
#종속변수를 맨 오른쪽으로 위치 변경
검사=data_전처리[['잔욕염색 검사_L*','잔욕염색 검사_a*', '잔욕염색 검사_b*', '잔욕염색 검사_K/S']]
data_전처리=data_전처리.drop(['잔욕염색 검사_L*','잔욕염색 검사_a*', '잔욕염색 검사_b*', '잔욕염색 검사_K/S'],axis=1)
data_전처리=pd.concat([data_전처리,검사],axis=1)
data_전처리

,경사_소재명,경사_섬도,경사_필라멘트수,경사_광택,경사_특성,경사_밀도,경사_형상,경사_성상_섬도 #1,경사_성상_섬도 #2,경사_성상_섬도 #3,...,Dianix Blue AM-2G_prop,Dianix Black AM-SLR_prop,Dianix Grey AM-SLR_prop,Dianix Yellow Brown AM-R_prop,Dorosperse Yellow KKL_prop,Dorosperse Black KKL_prop,잔욕염색 검사_L*,잔욕염색 검사_a*,잔욕염색 검사_b*,잔욕염색 검사_K/S
0,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,48.17,63.13,15.55,13.4
1,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,47.96,63.42,15.77,13.9
2,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,50.06,62.44,13.65,11.1
3,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,49.47,62.67,14.05,11.8
4,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,52,61.27,11.83,8.872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3165,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,74.5775,4.785,-5.3225,0.33075
3166,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,75.47,5.175,-4.5425,0.3095
3167,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,74.86,5.12,-4.915,0.32575
3168,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,75.3325,4.73,-4.75,0.3095


# 중복데이터제거

In [28]:
ch=["a"]*len(data_전처리.columns)

In [29]:
nm=[]
for i in range(len(ch)):
    nm.append(ch[i]+str(i))
gan_columns=nm

In [30]:
origin_columns=data_전처리.columns

In [31]:
data_전처리.columns=gan_columns

In [32]:
data_pret=data_전처리

In [34]:
np.array(data_전처리.columns)

array(['a0', 'a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7', 'a8', 'a9', 'a10',
       'a11', 'a12', 'a13', 'a14', 'a15', 'a16', 'a17', 'a18', 'a19',
       'a20', 'a21', 'a22', 'a23', 'a24', 'a25', 'a26', 'a27', 'a28',
       'a29', 'a30', 'a31', 'a32', 'a33', 'a34', 'a35', 'a36', 'a37',
       'a38', 'a39', 'a40', 'a41', 'a42', 'a43', 'a44', 'a45', 'a46',
       'a47', 'a48', 'a49', 'a50', 'a51', 'a52', 'a53', 'a54', 'a55',
       'a56', 'a57', 'a58', 'a59', 'a60', 'a61', 'a62', 'a63', 'a64',
       'a65', 'a66', 'a67', 'a68', 'a69', 'a70', 'a71', 'a72', 'a73',
       'a74', 'a75', 'a76', 'a77', 'a78', 'a79', 'a80', 'a81', 'a82',
       'a83', 'a84', 'a85', 'a86', 'a87', 'a88', 'a89', 'a90', 'a91',
       'a92', 'a93', 'a94', 'a95', 'a96', 'a97', 'a98', 'a99', 'a100',
       'a101', 'a102', 'a103', 'a104', 'a105', 'a106', 'a107', 'a108',
       'a109', 'a110', 'a111', 'a112', 'a113', 'a114', 'a115', 'a116',
       'a117', 'a118', 'a119', 'a120', 'a121', 'a122', 'a123', 'a124',
       'a125

In [35]:
q='''
select a0, a1, a2, a3, a4, a5, a6, a7, a8, a9, a10,a11, a12, a13, a14, a15, a16, a17, a18, a19,a20, a21, a22, a23, a24, a25, a26, a27, a28,a29, a30, a31, a32, a33, a34, a35, a36, a37,a38, a39, a40, a41, a42, a43, a44, a45, a46,a47, a48, a49, a50, a51, a52, a53, a54, a55,a56, a57, a58, a59, a60, a61, a62, a63, a64,a65, a66, a67, a68, a69, a70, a71, a72, a73,a74, a75, a76, a77, a78, a79, a80, a81, a82,a83, a84, a85, a86, a87, a88, a89, a90, a91,a92, a93, a94, a95, a96, a97, a98, a99, a100,a101, a102, a103, a104, a105, a106, a107, a108,a109, a110, a111, a112, a113, a114, a115, a116,a117, a118, a119, a120, a121, a122, a123, a124,a125, a126, a127, a128, a129, a130, a131, a132,a133, a134, a135, a136, a137, a138, a139, a140,a141, a142, a143, a144, a145, a146, a147, a148,a149, a150, a151, a152, a153, a154, a155, avg(a156),avg(a157), avg(a158), avg(a159)
from data_pret
group by a0, a1, a2, a3, a4, a5, a6, a7, a8, a9, a10,a11, a12, a13, a14, a15, a16, a17, a18, a19,a20, a21, a22, a23, a24, a25, a26, a27, a28,a29, a30, a31, a32, a33, a34, a35, a36, a37,a38, a39, a40, a41, a42, a43, a44, a45, a46,a47, a48, a49, a50, a51, a52, a53, a54, a55,a56, a57, a58, a59, a60, a61, a62, a63, a64,a65, a66, a67, a68, a69, a70, a71, a72, a73,a74, a75, a76, a77, a78, a79, a80, a81, a82,a83, a84, a85, a86, a87, a88, a89, a90, a91,a92, a93, a94, a95, a96, a97, a98, a99, a100,a101, a102, a103, a104, a105, a106, a107, a108,a109, a110, a111, a112, a113, a114, a115, a116,a117, a118, a119, a120, a121, a122, a123, a124,a125, a126, a127, a128, a129, a130, a131, a132,a133, a134, a135, a136, a137, a138, a139, a140,a141, a142, a143, a144, a145, a146, a147, a148,a149, a150, a151, a152, a153, a154, a155
'''
q=sqldf(q)
#print(sqldf(q, locals()))

In [36]:
q.columns=origin_columns
q

,경사_소재명,경사_섬도,경사_필라멘트수,경사_광택,경사_특성,경사_밀도,경사_형상,경사_성상_섬도 #1,경사_성상_섬도 #2,경사_성상_섬도 #3,...,Dianix Blue AM-2G_prop,Dianix Black AM-SLR_prop,Dianix Grey AM-SLR_prop,Dianix Yellow Brown AM-R_prop,Dorosperse Yellow KKL_prop,Dorosperse Black KKL_prop,잔욕염색 검사_L*,잔욕염색 검사_a*,잔욕염색 검사_b*,잔욕염색 검사_K/S
0,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.67,0.0,93.21375,-0.67375,4.01250,0.058625
1,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.67,0.0,93.04875,-0.56375,4.22250,0.062750
2,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.67,0.0,93.37625,-0.49875,3.93000,0.058625
3,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.67,0.0,93.45375,-0.53125,3.87250,0.057375
4,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.67,0.0,93.52625,-0.57000,3.78375,0.056750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1539,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,96,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.00,0.0,63.39625,49.13875,5.32875,2.509250
1540,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,96,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.00,0.0,64.74375,47.87000,4.69125,2.197750
1541,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,96,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.00,0.0,77.35625,25.01500,2.79625,0.484125
1542,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,96,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.00,0.0,78.72250,22.50250,2.95500,0.403625


In [37]:
q.to_csv('C:/Users/leewa/Desktop/1월전처리데이터.csv',encoding='euc-kr')

In [ ]:
ㅔㅇ.채ㅜ